In [26]:
import pandas as pd
import numpy as np
from RBFNN import RBFNN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import glob

In [27]:
# 1. Ambil semua file CSV di folder saat ini yang sesuai pola
csv_files = glob.glob("*.csv")

# 2. Gabungkan semua file
df_list = [pd.read_csv(f) for f in csv_files]
df_all = pd.concat(df_list, ignore_index=True)

# Filter hanya baris dengan format filename yang benar (misal: 0_1.png)
df_all = df_all[df_all['filename'].str.contains(r'^\d+_\d+\.png$', regex=True)]

# Ekstrak label dari filename
df_all['label'] = df_all['filename'].str.extract(r'^(\d+)_')[0].astype(int)



# files = ['Kelas 1 40.csv', 'Kelas 2 40.csv', 'Kelas 3 40.csv', 'Kelas 4 40.csv']
# df_all = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)



In [28]:
X = df_all.drop(columns=['filename', 'label']).values.astype(float)
y = df_all['label'].values


In [29]:
# kita akan kumpulkan potongan per label:
X_train_parts, X_test_parts = [], []
y_train_parts, y_test_parts = [], []

# ambil kembali DataFrame agar mudah groupby:
df_all = pd.DataFrame(
    np.hstack([X, y.reshape(-1,1)]),
    columns=[f'pixel_{i}' for i in range(X.shape[1])] + ['label']
)

for lab, group in df_all.groupby('label'):
    # pisah fitur dan target utk setiap grup
    Xg = group.drop('label', axis=1).values.astype(float)
    yg = group['label'].values.astype(int)
    
    # split per grup
    Xg_train, Xg_test, yg_train, yg_test = train_test_split(
        Xg, yg,
        test_size=0.2,
        random_state=42,
        shuffle=True
    )
    
    X_train_parts.append(Xg_train)
    X_test_parts.append(Xg_test)
    y_train_parts.append(yg_train)
    y_test_parts.append(yg_test)

# gabungkan kembali semua grup
X_train = np.vstack(X_train_parts)
X_test  = np.vstack(X_test_parts)
y_train = np.hstack(y_train_parts)
y_test  = np.hstack(y_test_parts)


In [30]:
# 5. Latih model
model = RBFNN(n_centers=10)
model.fit(X_train, y_train)

# 6. Prediksi & evaluasi
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.085
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.00      0.00      0.00        20
           2       0.00      0.00      0.00        20
           3       0.00      0.00      0.00        20
           4       0.04      0.15      0.06        20
           5       0.12      0.70      0.21        20
           6       0.00      0.00      0.00        20
           7       0.00      0.00      0.00        20
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00        20

    accuracy                           0.09       200
   macro avg       0.02      0.08      0.03       200
weighted avg       0.02      0.09      0.03       200



c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
